# 조회에 필요한 데이터 불러오기

In [1]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [3]:
files = sorted(glob('./data/*.csv'))
date_cols = ['date']
data_all = pd.DataFrame()
for file in tqdm(files):
    df_day = pd.read_csv(file, parse_dates=date_cols) # date 컬럼 datetime 형식으로 불러오기
    data_all = pd.concat([data_all, df_day[['press', 'title', 'date', 'link']]])
data_all.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████| 730/730 [03:30<00:00,  3.46it/s]


In [4]:
data_all

,press,title,date,link
0,중앙일보,프로배구 2·3일 경기 연기.. 중계방송 스태프 코로나19 확진,2021-01-01,https://v.daum.net/v/20210101233343622
1,세계일보,"""산에서 커피 팔고 싶어"" 조두순의 소박한 꿈 이뤄질까? 시민들 '불안'",2021-01-01,https://v.daum.net/v/20210101233139613
2,서울신문,확진 1000명 목전 두고..秋 '구치소 집단감염' 첫 사과(종합),2021-01-01,https://v.daum.net/v/20210101233101608
3,국민일보,문건 유출? 미루기?.. 벌써 다 퍼진 '2.5단계 연장안',2021-01-01,https://v.daum.net/v/20210101231448533
4,뉴스1,'해변 폐쇄·드론 감시' 강릉 해맞이객 90% 줄어..그래도 1만6천명,2021-01-01,https://v.daum.net/v/20210101231238518
...,...,...,...,...
1411408,세계일보,"한국남동발전, 8년째 취약계층 평생학습 지원",2022-12-31,https://v.daum.net/v/20221231010122552
1411409,세계일보,"KIS제주, 하버드대 2년 연속 합격생 배출",2022-12-31,https://v.daum.net/v/20221231010117551
1411410,한국일보,늘어나는 방음터널... 번지면 대형 화재인데 방염 규정은 전무,2022-12-31,https://v.daum.net/v/20221231001003411
1411411,국민일보,[핫인픽] #배달할증료 #한파·폭설 출근길 #메시 지폐,2022-12-31,https://v.daum.net/v/20221231000423387


# 전체 데이터 유사도 추천

## 모델 불러오기

In [7]:
import pickle

In [8]:
with open("./model/pickled_tfidf_vec.bin", "rb") as f:
    tfidf_vec = pickle.load(f)

In [9]:
with open("./model/pickled_tfidf_dtm.bin", "rb") as f:
    tfidf_dtm = pickle.load(f)

## 형태소 추출

In [10]:
def string_tokenizer(content):
    import pandas as pd
    from konlpy.tag import Okt
    okt = Okt()
    
    content = pd.Series(content)
    # 한글, 숫자, 공백만 남기기
    content = content.str.replace('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)
    token_content = okt.pos(content[0], norm=True, stem=True)
    return token_content

In [11]:
except_list = ['Exclamation', 'Josa', 'KoreanParticle', 'Determiner',  'Eomi', 'Suffix',  'VerbPrefix', 'PreEomi']
include_list = ['Verb',  'Noun']

In [12]:
# 230207 기현
def tag_except_list(except_list, token_content):
    filtered_list= []
    for tag in token_content :
        if tag[1] in except_list:
            continue
        filtered_list.append(tag[0])
    content = " ".join(filtered_list)
    return content

In [13]:
# 230207 기현
def tag_include_list(include_list, token_content):
    filtered_list=[]
    for tag in token_content :
        if tag[1] in include_list:
            filtered_list.append(tag[0])

    content = " ".join(filtered_list)
    return content

## 크롤링된 기사 제목으로 유사 기사 추천

In [14]:
def get_recommend_by_title(title, n, drange=False):
    from sklearn.metrics.pairwise import cosine_similarity
    # 타이틀로 저장된 토큰 호출, 축적된 기사들과 유사도 연산
    token = data_all.loc[data_all['title'] == title]['okt'].values[0]
    tfidf_content = tfidf_vec.transform([token])
    
    # 기간 설정
    if drange:
        data_filtered_idx = data_all[data_all['date'].isin(pd.date_range(drange[0], drange[1]))].index
        tfidf_dtm_filtered = tfidf_dtm[data_filtered_idx]
        cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm_filtered)
    else:
        cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm)
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1], 4)*100 for sim in sim_scores_n]
        
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [ ]:
title = "19년 동안 2억2000만원 내 놓은 기부천사 황영희씨"
data_all.loc[data_all['title'] == title]['link']

In [15]:
%%time
get_recommend_by_title("19년 동안 2억2000만원 내 놓은 기부천사 황영희씨", 10)

KeyError: 'okt'

In [ ]:
%%time
get_recommend_by_title('19년 동안 2억2000만원 내 놓은 기부천사 황영희씨', 10, drange=['2022-01-01', '2022-12-31'])

## 다음 url로 유사 기사 추천

In [16]:
stop_content = ["무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]
rep_list = ['기사내용 요약']

In [17]:
def create_soup(url):
    from bs4 import BeautifulSoup
    import requests
    i = 0
    ## 요청 오류시 10번  재시도
    while i < 10 :
        try:
            res = requests.get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
            break
        except:
            i += 1
            
    return soup

In [18]:
def content_only_scraper(link):
    article_soup = create_soup(link)

    # 본문 정리
    article = article_soup.find_all('section')[1]
    content = article.find_all(True, attrs={'dmcf-ptype':'general'})
    rst = []
    for para in content:
        for tmp in para.text.split('\n'):
            if tmp.strip() != '':
                rst.append(tmp.strip())
    content = []
    for c in rst:
        for i in stop_content:
            if i in c:
                break
        else:
            for rep in rep_list:
                c = c.replace(rep, '')
            content.append(c)
    content = ' '.join(content)
    if len(content) < 200:
        return False
    
    return content

In [19]:
def get_recommend_by_url(url, n, drange=False):
    from sklearn.metrics.pairwise import cosine_similarity
    # 다음 뉴스인지 확인
    if not url.startswith('https://v.daum.net/v/'):
        return False
    # 본문 스크랩
    content = content_only_scraper(url)

    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    tfidf_content = tfidf_vec.transform([token])
    
    # 기간 설정
    if drange:
        data_filtered_idx = data_all[data_all['date'].isin(pd.date_range(drange[0], drange[1]))].index
        tfidf_dtm_filtered = tfidf_dtm[data_filtered_idx]
        cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm_filtered)
    else:
        cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm)
    
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1], 4)*100 for sim in sim_scores_n]
    
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [21]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10)

CPU times: total: 1min 6s
Wall time: 1min 55s


,press,similarity,title,date,link
252178,국민일보,30.28,조국 재판 6개월 만에 재개..정경심과 법정 같이 설듯,2021-04-30,https://v.daum.net/v/20210430140850136
339634,문화일보,30.06,'피고인' 조국·정경심 나란히 법정 출석,2021-06-11,https://v.daum.net/v/20210611112009228
252388,뉴시스,28.49,"조국·정경심, 6월11일 법정 같이선다..반년만 재판 재개",2021-04-30,https://v.daum.net/v/20210430121651371
1357317,세계일보,28.45,‘자녀 입시 비리·감찰 무마’ 조국 전 장관에 징역 5년 구형,2022-12-02,https://v.daum.net/v/20221202162327923
1357592,연합뉴스,28.45,"검찰 ""조국, 명백한 사실도 인정 안 해""…징역 5년 구형(종합)",2022-12-02,https://v.daum.net/v/20221202145347140
1357066,연합뉴스,28.35,"징역 5년 구형받은 조국 ""검찰의 무차별 공격에 무력""(종합2보)",2022-12-02,https://v.daum.net/v/20221202183710978
1358639,뉴시스,28.32,'입시비리'·감찰무마' 조국 재판 오늘 결심…3년 만에 마무리 수순,2022-12-02,https://v.daum.net/v/20221202060032795
327314,뉴시스,28.17,"조국·정경심, 나란히 법정 선다..6개월만에 재판 재개",2021-06-06,https://v.daum.net/v/20210606050052336
1357681,뉴시스,27.97,"'자녀 입시비리' 조국 전 장관, 1심 결심공판 출석 [뉴시스Pic]",2022-12-02,https://v.daum.net/v/20221202141500738
1346082,뉴시스,27.96,"조국 입시비리 등 의혹, 이번주 결심…이르면 올해 1심 결론",2022-11-27,https://v.daum.net/v/20221127095726043


## 직접 입력한 본문(문자열)로 유사 기사 추천

In [ ]:
def get_recommend_by_content(content, n, drange=False):
    from sklearn.metrics.pairwise import cosine_similarity
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    
    tfidf_content = tfidf_vec.transform([token])
    
    # 기간 설정
    if drange:
        data_filtered_idx = data_all[data_all['date'].isin(pd.date_range(drange[0], drange[1]))].index
        tfidf_dtm_filtered = tfidf_dtm[data_filtered_idx]
        cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm_filtered)
    else:
        cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm)
    
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1], 4)*100 for sim in sim_scores_n]
        
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [ ]:
%%time
arc_content = """19년 간 2억원이 넘는 기부를 실천한 사람이 있다.

지난 2004년부터 매년 꾸준히 어려운 이웃을 위한 나눔을 실천, 19년간 2억 2000만원을 기부한 시민이 있어 지역사회의 귀감이 되고 있다.

코로나19가 길어져 취약 계층의 복지 수요가 늘었다고 판단한 황 대표는 올해 1월부터는 기부액을 매달 200만원으로 늘렸다.

이렇게 이어온 황 대표의 누적 기부 금액은 2억 2000만원에 달한다."""
get_recommend_by_content(arc_content, 10)

In [ ]:
%%time
arc_content = """빅토르 안 국내 복귀 시도"""
get_recommend_by_content(arc_content, 10)